# Alumno: Gerardo de Miguel González

## Keras Prediccion

**Practica 2** - predecir la precipitación en Madrid. Hay que entregar tanto el archivo con la prediccion (.rda) como el codigo que lo genera (.html a ser posible, si lo haceis en jupyter notebooks lo podeis guardar como .html).

 1) Si habéis predicho la ocurrencia de precipitacion y la cantidad de precipitacion por separado entonces enviadme ambas predicciones tal que:

```r
save(yTest_ocu,yTest_reg,file="yTest.rda")
```

 2) Si habeis predicho la ocurrencia y la cantidad de precipitation con el mismo modelo entonces enviadmelo asi:

```r
save(yTest,file="yTest.rda")
```

## Descripción detallada

El objetivo es diseñar una red neuronal que sea capaz de predecir el valor de la precipitación mediante la información de variables de larga escala. El dataset consiste en 450 predictores y un predictando, la precipitación en Madrid.

Hay que tener en cuenta que la precipitación es una variable mixta, ya que habrá días con un valor exacto de 0 y otros que se encuentren en el intervalo (0,inf).

Así pues, diseñar una/s red neuronal/es con KERAS, que sea capaz de predecir la precipitación de acuerdo con los siguientes índices:

  - La ocurrencia de precipitación se evalúa con el AUC.
  - La cantidad de precipitación se evalúa con el MSE.

Podréis valeros de todas las técnicas aprendidas hasta ahora: early stopping, regularización, backpropagation + momento, ajustar el learning rate, poner varias capas y demás. Esto no quiere decir que la red que contenga toda esta variedad de técnicas será la que obtenga un menor error. Tenéis que jugar y probar distintos diseños de redes con el fin de encontrar la que consideréis que obtendría un menor error en un dataset distinto (es decir, que tenga buena
capacidad de generalización).

Construiréis vuestro modelo usando xTrain e yTrain, y después evaluaréis el modelo en xTest. La predicción que resulte al evaluar el modelo en xTest es lo que tenéis que guardar en un fichero y lo que se envía para corrección:

```r
save(yTest, file = “yTest.rda”)
```

**NOTA**: se considera día de lluvia si el valor de precipitación es mayor que 1, sino convertir a 0.

## Dataset Meteo Madrid

Se proporciona el dataset `Madrid_Alumno.rda` junto con el enunciado de la práctica

**::GMG::** ¿Cómo se lee el rda?

In [1]:
file.info('data/Madrid_Alumno.rda')

,size,isdir,mode,mtime,ctime,atime,uid,gid,uname,grname
data/Madrid_Alumno.rda,20099579,FALSE,644,2019-03-22 11:31:26,2019-03-22 11:31:26,2019-03-22 11:35:08,1000,100,jovyan,users


In [2]:
# https://stat.ethz.ch/R-manual/R-devel/library/base/html/load.html
load('data/Madrid_Alumno.rda')
ls()

[1] "xTest"  "xTrain" "yTrain"

In [3]:
str(xTest)

 num [1:2192, 1:450] 57168 56932 56256 55960 55665 ...


In [4]:
str(xTrain)

 num [1:8766, 1:450] 57043 56963 56523 54628 53584 ...


In [5]:
str(yTrain)

 num [1:8766(1d)] 0.2 0 7.4 0 0 1.6 7.8 0 0 0 ...
 - attr(*, "dimensions")= chr "time"
